<a href="https://colab.research.google.com/github/RaveeMishra/Sentiment_analysis/blob/main/LSTM_GRU_insta_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile requirements.txt

numpy==1.26.4
scikit-learn==1.4.2
torch==2.2.2
transformers==4.41.2
datasets==2.18.0
tqdm
pandas

Overwriting requirements.txt


In [8]:
pip install -r requirements.txt

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("INSTAGRAM_REVIEWS.csv")

# Keep only needed columns
df = df[["review_text", "review_rating"]]

# Remove missing values
df = df.dropna()

# Create sentiment label
def create_label(rating):
    if rating >= 4:
        return 1  # Positive
    elif rating <= 2:
        return 0  # Negative
    else:
        return None  # Neutral (we remove)

df["label"] = df["review_rating"].apply(create_label)

# Remove neutral reviews
df = df.dropna(subset=["label"])

# Rename review_text to text (to match your training code)
df = df.rename(columns={"review_text": "text"})

# Convert label to integer
df["label"] = df["label"].astype(int)

# Split (80% train / 20% test)
train_df, test_df = train_test_split(
    df[["text", "label"]],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# Save files
train_df.to_csv("instagram_train.csv", index=False)
test_df.to_csv("instagram_test.csv", index=False)

print("Dataset prepared & split successfully ✔")
print("Train size:", len(train_df))
print("Test size:", len(test_df))
print("\nLabel distribution (Train):")
print(train_df["label"].value_counts())


Dataset prepared & split successfully ✔
Train size: 2339688
Test size: 584923

Label distribution (Train):
label
1    1767852
0     571836
Name: count, dtype: int64


In [45]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from transformers import AutoTokenizer

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , accuracy_score
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm
import numpy as np

############################################
# CONFIG
############################################

MODEL_NAME = "distilbert-base-uncased"
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-4
MAX_LEN = 128

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)



############################################
# DATASET
############################################

class InstagramSentimentDataset(Dataset):

    def __init__(self, csv_path):

        df = pd.read_csv(csv_path)

        self.texts = df["text"].tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):

        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long)
        }


def get_loader(csv_path, shuffle=True):

    return DataLoader(
        InstagramSentimentDataset(csv_path),
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=2,
        pin_memory=True
    )

############################################
# MODELS
############################################

class SimpleLSTMClassifier(nn.Module):
    """
    Single-layer, Unidirectional LSTM
    """

    def __init__(self, vocab=30522, embed=128, hidden=256):
        super().__init__()

        self.embedding = nn.Embedding(vocab, embed)

        self.lstm = nn.LSTM(
            input_size=embed,
            hidden_size=hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden, 2)

    def forward(self, ids):

        x = self.embedding(ids)

        _, (hidden, _) = self.lstm(x)

        hidden = hidden[-1]  # Last layer hidden state

        hidden = self.dropout(hidden)

        return self.fc(hidden)


class DeepLSTMClassifier(nn.Module):
    """
    Multi-layer (Deep) LSTM
    """

    def __init__(self, vocab=30522, embed=128, hidden=256, layers=3):
        super().__init__()

        self.embedding = nn.Embedding(vocab, embed)

        self.lstm = nn.LSTM(
            input_size=embed,
            hidden_size=hidden,
            num_layers=layers,      # 🔥 Multiple stacked layers
            batch_first=True,
            bidirectional=True,     # Deep + Bidirectional
            dropout=0.3             # Applied between layers
        )

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden * 2, 2)

    def forward(self, ids):

        x = self.embedding(ids)

        _, (hidden, _) = self.lstm(x)

        # Concatenate last forward + backward states
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)

        hidden = self.dropout(hidden)

        return self.fc(hidden)


In [46]:
############################################
# MODELS (GRU VERSION)
############################################

class SimpleGRUClassifier(nn.Module):
    """
    Single-layer, Unidirectional GRU
    """

    def __init__(self, vocab=30522, embed=128, hidden=256):
        super().__init__()

        self.embedding = nn.Embedding(vocab, embed)

        self.gru = nn.GRU(
            input_size=embed,
            hidden_size=hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden, 2)

    def forward(self, ids):

        x = self.embedding(ids)

        _, hidden = self.gru(x)

        hidden = hidden[-1]   # Last layer hidden state

        hidden = self.dropout(hidden)

        return self.fc(hidden)


class DeepGRUClassifier(nn.Module):
    """
    Multi-layer (Deep) GRU - NOT bidirectional
    """

    def __init__(self, vocab=30522, embed=128, hidden=256, layers=3):
        super().__init__()

        self.embedding = nn.Embedding(vocab, embed)

        self.gru = nn.GRU(
            input_size=embed,
            hidden_size=hidden,
            num_layers=layers,   # 🔥 Stacked layers
            batch_first=True,
            bidirectional=False,
            dropout=0.3          # Applied between layers
        )

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden, 2)

    def forward(self, ids):

        x = self.embedding(ids)

        _, hidden = self.gru(x)

        hidden = hidden[-1]  # Last layer final hidden state

        hidden = self.dropout(hidden)

        return self.fc(hidden)




In [47]:
def train(model_type="simple"):
    train_loader = get_loader("instagram_train.csv", shuffle=True)
    test_loader = get_loader("instagram_test.csv", shuffle=False)

    if model_type == "simple":
        model = SimpleGRUClassifier().to(DEVICE)

    elif model_type == "deep":
        model = DeepGRUClassifier().to(DEVICE)

    else:
        raise ValueError("Choose 'simple' or 'deep'")


In [48]:

    print("Device:", DEVICE)

    # Simple GRU
    train(model_type="simple")

    print('*' * 100)

    # Deep GRU
    train(model_type="deep")

Device: cuda
****************************************************************************************************


In [ ]:

    print("Device:", DEVICE)

    # ----------------------------
    # Simple LSTM
    # ----------------------------
    train_loader = get_loader("instagram_train.csv", shuffle=True)
    test_loader = get_loader("instagram_test.csv", shuffle=False)

    simple_lstm_model = SimpleLSTMClassifier().to(DEVICE)
    optimizer = torch.optim.AdamW(simple_lstm_model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    # Training loop (same as before)
    for epoch in range(EPOCHS):
        simple_lstm_model.train()
        loop = tqdm(train_loader)
        for batch in loop:
            ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            logits = simple_lstm_model(ids)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f"Simple LSTM | Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())

        # Evaluate after each epoch using your metrics function
        evaluate(simple_lstm_model, test_loader, model_name="Simple LSTM")

    torch.save(simple_lstm_model.state_dict(), "simple_lstm_instagram.pt")
    print("Simple LSTM saved ✔\n")
    print('*' * 100)

    # ----------------------------
    # Deep LSTM
    # ----------------------------
    deep_lstm_model = DeepLSTMClassifier().to(DEVICE)
    optimizer = torch.optim.AdamW(deep_lstm_model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        deep_lstm_model.train()
        loop = tqdm(train_loader)
        for batch in loop:
            ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            logits = deep_lstm_model(ids)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f"Deep LSTM | Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())

        # Evaluate after each epoch
        evaluate(deep_lstm_model, test_loader, model_name="Deep LSTM")

    torch.save(deep_lstm_model.state_dict(), "deep_lstm_instagram.pt")
    print("Deep LSTM saved ✔\n")


Device: cuda


Simple LSTM | Epoch 1: 100%|██████████| 73116/73116 [19:44<00:00, 61.73it/s, loss=0.106]



Evaluation Metrics for Simple LSTM
Precision  : 0.8936
Recall     : 0.9595
F1         : 0.9253
ROC-AUC    : 0.8905
Accuracy   : 0.8830



Simple LSTM | Epoch 2: 100%|██████████| 73116/73116 [19:28<00:00, 62.56it/s, loss=1.03]



Evaluation Metrics for Simple LSTM
Precision  : 0.8998
Recall     : 0.9554
F1         : 0.9268
ROC-AUC    : 0.8936
Accuracy   : 0.8859



...

Deep LSTM | Epoch 1: 100%|██████████| 73116/73116 [57:40<00:00, 21.13it/s, loss=0.253]



Evaluation Metrics for Deep LSTM
Precision  : 0.9003
Recall     : 0.9519
F1         : 0.9254
ROC-AUC    : 0.8916
Accuracy   : 0.8840



Deep LSTM | Epoch 2: 100%|██████████| 73116/73116 [57:55<00:00, 21.04it/s, loss=0.0633]



Evaluation Metrics for Deep LSTM
Precision  : 0.8963
Recall     : 0.9605
F1         : 0.9273
ROC-AUC    : 0.8938
Accuracy   : 0.8862



Deep LSTM | Epoch 3:  14%|█▍        | 10135/73116 [08:03<49:12, 21.33it/s, loss=0.247]

In [40]:
def train(model_type="simple"):

    train_loader = get_loader("instagram_train.csv", shuffle=True)
    test_loader = get_loader("instagram_test.csv", shuffle=False)

    # Select model
    if model_type == "simple":
        model = SimpleGRUClassifier().to(DEVICE)
        model_name = "Simple GRU"

    elif model_type == "deep":
        model = DeepGRUClassifier().to(DEVICE)
        model_name = "Deep GRU"

    else:
        raise ValueError("Choose 'simple' or 'deep'")

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):

        model.train()
        loop = tqdm(train_loader)

        for batch in loop:

            ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            logits = model(ids)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f"{model_name} | Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item())

        # Evaluate after each epoch
        evaluate(model, test_loader, model_name)

    torch.save(model.state_dict(), f"{model_type}_instagram_sentiment.pt")
    print(f"\n{model_name} saved ✔\n")


In [41]:
def compute_metrics(logits, labels):

    probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    return {
        "Precision": precision_score(labels, preds),
        "Recall": recall_score(labels, preds),
        "F1": f1_score(labels, preds),
        "ROC-AUC": roc_auc_score(labels, probs),
        "Accuracy": accuracy_score(labels, preds)
    }



In [42]:
def evaluate(model, loader, model_name="Model"):

    model.eval()

    all_logits = []
    all_labels = []

    with torch.no_grad():
        for batch in loader:
            ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            logits = model(ids)

            all_logits.append(logits)
            all_labels.append(labels)

    # Concatenate all batches
    logits = torch.cat(all_logits)
    labels = torch.cat(all_labels)

    # Compute metrics using your original function
    metrics = compute_metrics(logits, labels)

    # Print metrics nicely
    print("\n" + "="*50)
    print(f"Evaluation Metrics for {model_name}")
    print("="*50)
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name:10s} : {metric_value:.4f}")
    print("="*50 + "\n")

    return metrics


In [43]:
    print("Device:", DEVICE)

    # Simple GRU
    train(model_type="simple")

    print('*' * 100)

    # Deep GRU
    train(model_type="deep")


Device: cuda


Simple GRU | Epoch 1:  21%|██        | 15153/73116 [04:31<17:17, 55.86it/s, loss=0.427]


KeyboardInterrupt: 